# 当前系统的操作流程及最终输出结果

## web页面操作(仅用于演示)

使用flask构建，运行方式如下：

```bash
python run.py
```

## 添加监管直播链接

在 DouyinLiveRecorder/config/URL_config.ini 文件中, 每一行对应一个直播链接，如下所示：

```text
https://live.douyin.com/815349576986,主播: VC官方旗舰店
https://live.douyin.com/484581718147,主播: 深海鱼油源头工厂
……
```

新增监管直播链接，直接新起一行加入链接即可，主播名称会在爬取过程中自动补全。

## 命令行运行方法

在本系统中，使用了tmux进行后台进程管理。因此在执行以下脚本前，需确保系统中已安装tmux。

通过脚本运行整个系统的方法如下所示：

```bash
# 开启系统工作
. recorder_and_analyzer_start.sh

# 关闭系统工作
. recorder_and_analyzer_end.sh
```

如果仅需运行直播视频获取模块，如下所示：

```bash
# 进入直播视频获取模块文件夹：
cd DouyinLiveRecorder

# 开启直播视频获取模块工作
. live_recorder_start.sh

# 关闭直播视频获取模块工作
. live_recorder_end.sh
```

## 直播获取模块

直播获取模块对应 DouyinLiveRecorder。其使用了ffmpeg进行视频的录制，需确保环境中已安装ffmpeg。

直播获取模块的配置文件为：DouyinLiveRecorder/config/config.ini ，在其中可以更改直播获取的相关选项，包括直播保存路径，视频保存格式，视频分段时间等等。

直播链接存放文件为：DouyinLiveRecorder/config/URL_config.ini


## 商品分类模型的调用

In [3]:
from category_recognize import category_recognize

def test_category_recognize_by_history():
    test_example = [
        "999柚美保健品专卖店"
        ]
    from category_recognize import category_recognize_by_history
    for i in test_example:
        result = category_recognize_by_history.recognize_category_by_history(i)
        print(result)

def test_category_recognize_by_llm():
    test_example = [
        "抖音商城年货节 59.9元 89.9 358元 50g 抗波·面霜 抖音官方旗舰店 自然美椰要 有模有漾 全国 过敏 假 包邮 包退 赔四 ",
        "999柚美保健品专卖店",
        "999德勤器械专卖店 ",
    ]
    def test_spark():
        from category_recognize import category_recognize_by_llm
        for i in test_example:
            result = category_recognize_by_llm.category_recognize_with_spark(i)
            print(result)
    def test_llama():
        from category_recognize import category_recognize_by_llm
        for i in test_example:
            result = category_recognize_by_llm.category_recognize_with_llama(i)
            print(result)
    
    # print("spark测试结果:")
    # test_spark()

    print("llama测试结果")
    test_llama()

In [4]:
test_category_recognize_by_history()
test_category_recognize_by_llm()

保健品
llama测试结果


KeyboardInterrupt: 

## 直播违法行为匹配模块 （包含 变体词发现模块）

In [ ]:
# test sensitive_word_match
def test_sensitive_word_match():
    from match.sensitive_word_match import sensitive_word_match
    test_example = [
        "国家级的",
        "最厉害的"
    ]
    print("敏感词匹配测试样例为：",test_example)
    print("敏感词匹配测试结果为:")
    for e in test_example:
        result = sensitive_word_match.match_sensitive_word(e)
        print(result)

# test variant_match
def test_variant_match():
    def test_re_match():
        from match.variant_match import re_match
        test_example = [
            "临某床",
            "鱼什么油",
            "最什么便宜的",
            "白某障"
        ]
        print("基于正则表达式的变体词匹配测试样例为：",test_example)

        print("基于正则表达式的变体词匹配测试，不要求变体词原词属于通用敏感词，测试结果为：")
        for i in test_example:
            result = re_match.detect_complex_variant_words(i)
            print(result)

        print("基于正则表达式的变体词匹配测试，要求变体词原词属于通用敏感词，测试结果为：")
        for i in test_example:
            result = re_match.detect_complex_variant_words_in_sensitive(i)
            print(result)

    def test_correct_match():
        import time
        cur_time = time.time()
        from match.variant_match import corrector_match
        print("引入corrector_match消耗时间为:", time.time() - cur_time)
        cur_time = time.time()

        test_example = [
            "这种药物的林床笑果非常好",
            "咱们医院里的蚁生都在推荐啊",
            "咱们的欣闹学管疾病等等都是可以治疗啊"
        ]
        print("基于纠错模型的变体词匹配测试样例为：",test_example)
        cur_time = time.time()
        print("基于纠错模型的变体词匹配测试样例测试结果为：")
        for i in test_example:
            result = corrector_match.match(i)
            print(result)
        print("运行kenlm_match,跑完所有测试样例,消耗时间为:", time.time() - cur_time)
    # test_re_match()
    def test_llm_match():
        import time
        cur_time = time.time()
        from match.variant_match import llm_match
        test_example = [
            "这种药物的林床笑果非常好",
            "咱们医院里的蚁生都在推荐啊",
            "咱们的欣闹学管疾病等等都是可以治疗啊",
            "如果不是吃了我们的东西的话啊，他第二天就得去医院找白大褂了, 可见我们的产品具有显著的临某床意义, 对我们的心脑血某管都是很有好处的"
        ]
        print("基于大模型的变体词匹配测试样例为：",test_example)
        cur_time = time.time()
        print("基于大模型的变体词匹配测试结果为：")
        for i in test_example:
            result = llm_match.variant_word_match_with_llama(i)
            print(result)
        print("调用大模型，跑完所有测试样例，消耗时间为：", time.time()-cur_time)
    
    test_re_match()
    # test_correct_match()
    # test_llm_match()

def test_ban_sale_match():
    from match.ban_sale_match import ban_sale_match
    test_example = [
        "国家级的马家窑",
        "春秋时期的瓷器",
        "电捕鱼器材",
        "香烟"
    ]
    print("敏感词匹配测试样例为：",test_example)
    print("敏感词匹配测试结果为:")
    for e in test_example:
        result = ban_sale_match.match_ban_limit_goods(e)
        print(result)

def test_goods_match():
    from match.goods_match import goods_conflict_match
    # print(goods_conflict_match.药品_merged)
    # print(goods_conflict_match.保健品_merged)
    # print(goods_conflict_match.化妆品_merged)
    # print(goods_conflict_match.医疗器械_merged)

    test_example = [
        "感冒清片纯中草药制作阿，不含有任何化学成分",
        "感冒清片非常有用",
        "香烟"
    ]
    print("商品矛盾性检测测试样例为：",test_example)
    print("敏感词匹配测试结果为:")
    for e in test_example:
        result = goods_conflict_match.match_goods_conflict_match(e)
        print(result)

In [ ]:
test_sensitive_word_match()

敏感词匹配测试样例为： ['国家级的', '最厉害的']
敏感词匹配测试结果为:
{'type': 1, 'content': {'通用敏感词': '国家'}}
{'type': 0, 'content': None}


In [ ]:
    # test_sensitive_word_match()
    test_variant_match()
    # test_ban_sale_match()
    # test_goods_match()

基于正则表达式的变体词匹配测试样例为： ['临某床', '鱼什么油', '最什么便宜的', '白某障']
基于正则表达式的变体词匹配测试，不要求变体词原词属于通用敏感词，测试结果为：
{'变体词': '临某床', '原词': '临床'}
{'变体词': '鱼什么油', '原词': '鱼油'}
{'变体词': '最什么便', '原词': '最便'}
{'变体词': '白某障', '原词': '白障'}
基于正则表达式的变体词匹配测试，要求变体词原词属于通用敏感词，测试结果为：
{'变体词': '临某床', '原词': '临床'}
None
{'变体词': '最什么便宜', '原词': '最便宜'}
None


In [ ]:
test_ban_sale_match()

敏感词匹配测试样例为： ['国家级的马家窑', '春秋时期的瓷器', '电捕鱼器材', '香烟']
敏感词匹配测试结果为:
{'type': 3, 'content': {'类型': '限售', '商品': '马家窑'}}
{'type': 3, 'content': {'类型': '限售', '商品': '春秋'}}
{'type': 3, 'content': {'类型': '限售', '商品': '捕鱼器'}}
{'type': 3, 'content': {'类型': '禁售', '商品': '香烟'}}


In [ ]:
test_goods_match()

商品矛盾性检测测试样例为： ['感冒清片纯中草药制作阿，不含有任何化学成分', '感冒清片非常有用', '香烟']
敏感词匹配测试结果为:


AttributeError: 'NoneType' object has no attribute 'encode'